Hypothesis: CSV will load into a simple schema if we remove all rows where the first column (an ID) is not an integer.

In [8]:
import pandas as pd
import numpy as np
import csv
from collections import defaultdict, Counter

In [5]:
schema_rows = []
with open("AAS_Chemical_Data_12202021.xlsx - Sheet1.csv", "r") as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if i == 0:
            header = row
        else:
            try:
                group_id = int(row[0])
            except:
                # ignore line
                pass
            else:
                schema_rows.append(row)

In [6]:
# pre-duplicates
len(schema_rows)

64162

In [11]:
len(header)

85

In [12]:
for field in header:
    print(field)

group_rid
GroupName
site_rid
SiteName
SiteLocation
event_rid
event_date
volunteer_time
data_entry
participants
rain_24_hours
weather
rain_hours
rain_inches
distance
air_temperature
stream
method
wqi
habitat
Comments
createdby
createddate
chem_detail_rid
air_temp
water_temp
calibrate
calibrate_comment
chemical_comment
ChemWarnings
do_saturation
reagent
reagent_other
ph1
ph2
DissolvedOxygen1
DissolvedOxygen2
Conductivity
Salinity1
Salinity2
SecchiDisk1
SecchiDisk2
ChlorophyllA
Alkalinity
AmmoniaN
NitrateN
Orthophosphate
SamplingDepth
SettleableSolids
Turbidity
Chloride
Hardness
Other1
Other2
Other3
Other4
Other5
Other6
Other7
Other1_Comm
Other2_Comm
Other3_Comm
Other4_Comm
Other5_Comm
Other6_Comm
Other7_Comm
bact_detail_rid
plate_blank
plate_one
plate_two
plate_three
plate_four
plate_five
colony_avg
hold_start_datetime
hold_end_datetime
min_temp
max_temp
three_M_plate
ecoli_idexx
fecal_coliform
ecoli_other
ecoli_other_unit
comments
warnings


In [6]:
# this is only a simple-minded attempt to remove duplicates (see below)
data = pd.DataFrame(data=schema_rows, columns=header).drop_duplicates()

In [55]:
len(data)

64148

## Validation of schema

## Dates

Observation: event dates use two-digit year and seem to start in the late '90s, so to order the values correctly we have to convert to a four-digit year.

In [198]:
data = data.assign(event_date=pd.to_datetime(data["event_date"]))

## Step 1

Convert NULLs ID columns (only occurs in `bact_detail_rid`) to -1 and make these columns integers

In [187]:
all_id_cols = ["group_rid", "site_rid", "event_rid", "chem_detail_rid", "bact_detail_rid"]

In [199]:
for id_col in all_id_cols:
    data.loc[data[id_col].isin(('NULL', '')), id_col] = '-1'
    data = data.assign(**{id_col: data[id_col].astype('int32')})

Check for duplicates, noting that some auto-generated IDs might still increment while all other user-entered data might be the same.

In [59]:
# these are unexpectedly non-unique
# - is the whole row a dupe?
len(set(data["event_rid"])) == len(data["event_rid"].values)

False

In [60]:
data["event_rid"].value_counts()[:30]

62854    12
92294     4
66076     4
78928     4
60150     4
75854     3
70773     3
88459     3
87651     3
59756     3
89389     3
70281     3
70030     3
69275     3
64432     3
82551     3
83275     3
83193     3
66028     3
89529     2
57775     2
62716     2
71526     2
64190     2
83607     2
59521     2
66312     2
83196     2
59819     2
62858     2
Name: event_rid, dtype: int64

In [148]:
def show_diffs(row1, row2):
    for i, (x1, x2) in enumerate(zip(row1, row2)):
        if x1 != x2:
            print(i, x1, "|||", x2)

From manual review of remaining dupes, it appears that there are always only two non-duped records after ignoring chem and bact detail rid's. It looks like the second entry either has a slightly different value to the first in one column, or there's a slew of additions in the second that are blank in the first.

This suggests the second entry is a correction to the first, so we default to always choosing the second.

In [156]:
bad_indices = []
for event_rid, cnt in data["event_rid"].value_counts().items():
    if cnt == 1:
        break
    test = data[data["event_rid"] == event_rid]
    test = test.drop(columns=["chem_detail_rid", "bact_detail_rid"])
    new_test = test.drop_duplicates()
    # do special cases first
    if len(new_test) == 2:
        # keep the second (see comments above)
        bad_indices.append(new_test.index[0])
    else:
        # use this for manual review of dupe instances
        if len(new_test) != 1:
            print(event_rid, cnt)
            print(len(new_test))
            print(list(new_test.index))
            show_diffs(new_test.iloc[0], new_test.iloc[1])
            if len(new_test) > 2:
                show_diffs(new_test.iloc[1], new_test.iloc[2])
            print("==============")
    bad_indices.extend(list(test.index[1:]))

In [200]:
deduped_data = data.drop(index=bad_indices)

In [190]:
len(deduped_data)

63951

In [191]:
# verify no more dupes in event_rid
deduped_data["event_rid"].value_counts()[:5]

2047     1
80434    1
35396    1
45635    1
47682    1
Name: event_rid, dtype: int64

In [202]:
deduped_data.sort_values(by="event_date")

,group_rid,GroupName,site_rid,SiteName,SiteLocation,event_rid,event_date,volunteer_time,data_entry,participants,...,hold_end_datetime,min_temp,max_temp,three_M_plate,ecoli_idexx,fecal_coliform,ecoli_other,ecoli_other_unit,comments,warnings
8415,189,Streams Alive!,160,Long Island Creek,33.8965 -84.4093,25289,1995-01-01 15:00:00,60,3089,1,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
8416,189,Streams Alive!,160,Long Island Creek,33.8965 -84.4093,25290,1995-02-05 15:00:00,60,3089,1,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
13351,623,Higgins Family,378,Willeo Creek,34.0415 -84.4208,30629,1995-02-12 13:20:00,20,7539,1,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
8417,189,Streams Alive!,160,Long Island Creek,33.8965 -84.4093,25291,1995-03-12 15:30:00,60,3089,1,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
8418,189,Streams Alive!,160,Long Island Creek,33.8965 -84.4093,25292,1995-04-02 19:00:00,60,3089,1,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64135,2808,rock dove lagoon,6644,rock dove lagoon,32.0532 -81.2436,92289,2021-12-18 13:52:00,20,40742,1,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
64147,2803,"Tolomato Island, Darien",6596,Crum Creek,31.4245 -81.3861,92298,2021-12-18 15:15:00,20,40490,1,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
64155,2227,GSW Sciences,7175,Muckalee Creek at McLittle Bridge Rd,32.0067 -84.2266,92306,2021-12-19 07:25:00,45,36976,1,...,50:00.0,34.7,35.2,0,NULL,NULL,NULL,NULL,Very high counts but creek was in flood stage ...,
64146,1696,Hiwassee River Watershed Coalition,5000,Brasstown Creek at Misty Creek subdivision,34.984 -83.8886,92297,2021-12-19 10:20:00,160,41392,2,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [21]:
deduped_data.to_csv("export_dataframe_stage1.csv", index=False)

# 